In [1]:
import requests
from pprint import pprint
import re
import lxml.html
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def log_error(line1, line2):
    _line1 = str(line1)
    _line2 = str(line2)
    diff = 0
    chars1 = len(_line1) + 10
    chars2 = len(_line2) + 10
    if chars1 > chars2:
        diff = chars1 - chars2
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line2 = '# ' + _line2
        else:
            _line2 = ' ' + _line2
        main_hashes = ''.join(['#' for i in range(chars1)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes}{_line2} {hashes}\n"                    
                    f"{main_hashes}\n")
        return(log_line)
   
    elif chars2 > chars1:
        diff = chars2 - chars1
        if diff % 2 != 0:
            diff = int(diff - .5)
            _line1 = '# ' + _line1
        else:
            _line1 = ' ' + _line1
        main_hashes = ''.join(['#' for i in range(chars2)])
        hashes = ''.join(['#' for i in range(int(diff/2)+4)])
        log_line = (f"{main_hashes}\n"
                    
                    f"{hashes}{_line1} {hashes}\n"
                    f"#### {_line2} ####\n"
                    f"{main_hashes}\n")
        return(log_line)
        
    else:
        hashes = '####'
        main_hashes = ''.join(['#' for i in range(chars2)])
        log_line = (f"{main_hashes}\n"
                    f"#### {_line1} ####\n"
                    f"{hashes} {_line2} {hashes}\n"
                    f"{main_hashes}\n")
        return(log_line)
        

In [3]:
# init session 
S = requests.Session()

# set URL
URL = "https://en.wikipedia.org/w/api.php"

In [4]:
page_id = "23553357"
topic = 'street_names'
subtopic = 'san_jose'

In [5]:
# get sections
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "sections"
}

sectionsR = S.get(url=URL, params=PARAMS)
sectionDATA = sectionsR.json()



In [6]:
sectionDATA

{'parse': {'title': 'List of streets in San Jose, California',
  'pageid': 23553357,
  'sections': [{'toclevel': 1,
    'level': '2',
    'line': 'References',
    'number': '1',
    'index': '1',
    'fromtitle': 'List_of_streets_in_San_Jose,_California',
    'byteoffset': 26321,
    'anchor': 'References'},
   {'toclevel': 1,
    'level': '2',
    'line': 'External links',
    'number': '2',
    'index': '2',
    'fromtitle': 'List_of_streets_in_San_Jose,_California',
    'byteoffset': 26451,
    'anchor': 'External_links'}]}}

In [ ]:
sections = [sec['anchor'] for sec in sectionDATA["parse"]["sections"]]

In [7]:
# create section list filtering out non-data sections
filt = [
    'References',
    #'See_also',
    #'External_links'
]

incl = [
    'Schemes',
    'Tropes'
]

sec_list = [sec for sec in sectionDATA["parse"]["sections"] if not sec['anchor'] in filt]
# sec_list = [sec for sec in sectionDATA["parse"]["sections"] if sec['anchor'] in incl]


len(sec_list)
sec_list

[{'toclevel': 1,
  'level': '2',
  'line': 'Westminster',
  'number': '1',
  'index': '1',
  'fromtitle': 'Street_names_of_Westminster',
  'byteoffset': 685,
  'anchor': 'Westminster'},
 {'toclevel': 1,
  'level': '2',
  'line': 'Leicester Square area',
  'number': '2',
  'index': '2',
  'fromtitle': 'Street_names_of_Westminster',
  'byteoffset': 28595,
  'anchor': 'Leicester_Square_area'}]

In [ ]:
for index, line in enumerate(lines):
    print(index, line)

In [51]:
# create nested section_dict dictionary from section list based on toclevel
# refactored
section_dict = {}
for i in range(len(sec_list)):
    if i == 0:
        if sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[sec_list[i]['line']] = {}
            current_grandparent = sec_list[i]['line']
        else:
            section_dict[sec_list[i]['line']] = []

    elif i != len(sec_list) - 1:
        if sec_list[i - 1]['toclevel'] < sec_list[i]['toclevel'] and sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[current_grandparent][sec_list[i]['line']] = {}
            current_parent = section_dict[current_grandparent][sec_list[i]['line']]
            
        elif sec_list[i]['toclevel'] == sec_list[i + 1]['toclevel'] or sec_list[i]['toclevel'] > sec_list[i + 1]['toclevel']:
            current_parent[sec_list[i]['line']] = []
            
        elif sec_list[i]['toclevel'] < sec_list[i + 1]['toclevel']:
            section_dict[current_grandparent][sec_list[i]['line']] = {}
            current_parent = section_dict[current_grandparent][sec_list[i]['line']]
            
    else:
        if sec_list[i]['toclevel'] == sec_list[i - 1]['toclevel']:
            section_dict[sec_list[i]['line']] = []
        else:
            section_dict[sec_list[i]['line']] = []
              

#pprint(section_dict)

In [46]:
section_dict

{'San Jose': []}

In [52]:
# hard coded name for section in section dict bec inconsistent wikipedia format
section_dict = {'San Jose': []}

In [ ]:
# create nested section_dict dictionary from section list based on toclevel
# adding 'Z' section by hand consider refactor
section_dict = {}
is_new_parent = True
parent = ''
for i in range(len(sec_list) - 1):
    if sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 1:
        section_dict[sec_list[i]['anchor']] = []
    elif sec_list[i]['toclevel'] == 1 and sec_list[i + 1]['toclevel'] == 2:
        section_dict[sec_list[i]['anchor']] = {}
        parent = sec_list[i]['anchor']
    elif sec_list[i]['toclevel'] == 2:
        section_dict[parent][sec_list[i]['anchor']] = []
section_dict['Z'] = []
#section_dict

In [ ]:
# make list of top level sections
secs = [sec for sec, val in section_dict.items()]


In [ ]:
# make section dict if no nested sections
section_dict = {sec['anchor']:[] for sec in sec_list}

In [ ]:
# get links
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "links"
}

linkR = S.get(url=URL, params=PARAMS)
linkDATA = linkR.json()

pprint(linkDATA["parse"]["links"])

In [ ]:
links = [link['*'] for link in linkDATA["parse"]["links"]]
for i, link in enumerate(links):
    print(i, link)

In [ ]:
# visualize link data
for link in links[3:4]:
    # get page info
    PARAMS = {
        "action": "query",
        "titles": link,
        "format": "json",
        "prop": "info",
        'inprop': "displaytitle"
    }

    infoR = S.get(url=URL, params=PARAMS)
    infoDATA = infoR.json()
    pprint(infoDATA)

In [9]:
# get data
PARAMS = {
    "action": "parse",
    "pageid": page_id,
    "format": "json",
    "prop": "wikitext"
}

textR = S.get(url=URL, params=PARAMS)
textDATA = textR.json()


In [10]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')

In [28]:
for index, line in enumerate(lines[:155]):
    print(index, line)

0 {{refimprove|date=August 2017}}
1 This is a '''list of streets in [[San Jose, California]]''', with descriptions, historic significance, and name origins.
2 {{dynamic list}}
3 
4 * [[California State Route 85]], known in part as the West Valley Freeway and the Stevens Creek Freeway
5 * [[California State Route 87]], known in part as the Guadalupe Freeway
6 * [[California State Route 130]], known in part as Mount Hamilton Road and Alum Rock Avenue
7 * State Route 82, also known as [[California State Route 82|El Camino Real]], after the historic [[El Camino Real (California)|El Camino Real]]
8 * [[The Alameda (San Jose)|The Alameda]]&nbsp;— Spanish term for a public tree-lined avenue that translates in French to Boulevard. It refers to the stretch of historic road connecting Mission Santa Clara with the Pueblo of San Jose.
9 * Alum Rock Avenue&nbsp;— named after the misidentified [[Alum Rock Park|alum rock]] ([[Thenardite|thenardite mineral]]) and springs in the canyon at the end of Al

In [53]:
# parse 
#       WikiText
# data


debug = False
special = True

this_sec = 'San Jose'
this_item = ''


for index, line in enumerate(lines[:154]):
    #print(line)
    if debug: print("$$-3-$$")
    if re.match('\*\s', line[0:2]):
        #print(index)
        if debug: print("$$-4-$$")
        if '—' in line:
            print(f"$0$-{index}")
            this_item = re.search(r'\*\s(.*?)&nbsp;', line).group(1)
            section_dict[this_sec].append(this_item)
        else:
            print(f"$1$-{index}")
            print(line)
            if line[0:4] == '* [[': 
                this_item = re.search(r'\*\s\[\[(.*)\]\]', line).group(1)
            else:
                this_item = line.split(' ')[1]
            section_dict[this_sec].append(this_item)
            #this_item = re.search(r'\*\s(.*)', line).group(1)
        
                
                   
                    
    

$1$-4
* [[California State Route 85]], known in part as the West Valley Freeway and the Stevens Creek Freeway
$1$-5
* [[California State Route 87]], known in part as the Guadalupe Freeway
$1$-6
* [[California State Route 130]], known in part as Mount Hamilton Road and Alum Rock Avenue
$1$-7
* State Route 82, also known as [[California State Route 82|El Camino Real]], after the historic [[El Camino Real (California)|El Camino Real]]
$0$-8
$0$-9
$0$-10
$0$-11
$0$-12
$0$-13
$1$-14
* Balbach Street is named after John Balbach, owner of Pioneer Carriage Manufacturing in 1864.<ref name="ReferenceA"/>
$0$-15
$0$-16
$0$-17
$0$-18
$0$-19
$0$-20
$0$-21
$0$-22
$0$-23
$0$-24
$0$-25
$0$-26
$0$-27
$0$-28
$0$-29
$0$-30
$0$-31
$0$-32
$0$-33
$0$-34
$0$-35
$0$-36
$0$-37
$0$-38
$0$-39
$0$-40
$0$-41
$0$-42
$0$-43
$0$-44
$0$-45
$0$-46
$0$-47
$0$-48
$0$-49
$0$-50
$0$-51
$0$-52
$0$-53
$0$-54
$0$-55
$0$-56
$0$-57
$0$-58
$0$-59
$0$-60
$0$-61
$0$-62
$0$-63
$0$-64
$0$-65
$0$-66
$0$-67
$0$-68
$0$-69
$0$-70
$0$-71

In [54]:
section_dict

{'San Jose': ['California State Route 85',
  'California State Route 87',
  'California State Route 130',
  'State',
  '[[The Alameda (San Jose)|The Alameda]]',
  'Alum Rock Avenue',
  'Asbury',
  'Arroyo de Oro',
  'Auzerais Avenue',
  'Bailey Avenue',
  'Balbach',
  'Balboa Avenue',
  'Bascom Avenue',
  'Bassett Street',
  'Bernal Road',
  'Berryessa Road',
  'Bird Avenue',
  'Blach Place',
  'Blossom Hill Road',
  'Bollinger Road',
  'Booksin Avenue',
  'Branham Lane',
  'Cahill Street',
  'Camden Avenue',
  'Canoas Garden Avenue',
  'Chaboya Road',
  'Chapman Street',
  'Cinnabar Street',
  'Cleaves Avenue',
  'Cleveland Avenue',
  'Coe Avenue',
  'Cory Avenue',
  'Cottle Avenue (not to be confused with Cottle Road)',
  'Cottle Road',
  'Cunningham Avenue',
  'Curtner Avenue',
  'Dana Avenue',
  'Davis Street',
  'De Mattei Court',
  'Delmas Avenue (downtown San Jose)',
  'Di Fiore Drive',
  'Donohue Drive',
  'Dry Creek Road',
  'Eastus Drive',
  'Emory Street',
  'Fiesta Lane',
 

In [55]:
with open(f"../output/street_type_designations.txt", 'r+', encoding="utf-8") as filepath:
    doc = filepath.read()
street_types = doc.split('\n')
street_types

['Highway',
 'Autobahn',
 'Auto-estrada',
 'Autoroute',
 'Autostrada',
 'Autostrasse',
 'Bypass',
 'Expressway',
 'Freeway',
 'Motorway',
 'Pike',
 'Turnpike',
 'Avenue',
 'Boulevard',
 'Parade',
 'Road',
 'Street',
 'Arcade',
 'Alley',
 'Bay',
 'Branch',
 'Brook',
 'Burg',
 'Byway',
 'Camp',
 'Center',
 'Club',
 'Common',
 'Corner',
 'Course',
 'Dale',
 'Divide',
 'Drive',
 'Estate',
 'Flat',
 'Forge',
 'Fork',
 'Fort',
 'Gardens',
 'Gate',
 'Gateway',
 'Glen',
 'Green',
 'Grove',
 'Harbor',
 'Haven',
 'Heights',
 'Highlands',
 'Hollow',
 'Key',
 'Knoll',
 'Landing',
 'Lane',
 'Light',
 'Loaf',
 'Lock',
 'Lodge',
 'Manor',
 'Meadow',
 'Mews',
 'Mill',
 'Mission',
 'Neck',
 'Orchard',
 'Passage',
 'Path',
 'Pathway',
 'Ranch',
 'Rapid',
 'Rest',
 'Route',
 'Row',
 'Rue',
 'Run',
 'Station',
 'Terrace',
 'Throughway',
 'Trace',
 'Track',
 'Trafficway',
 'Trail',
 'Trailer',
 'Union',
 'Vale',
 'View',
 'Village',
 'Ville',
 'Vista',
 'Walk',
 'Wall',
 'Way',
 'Well',
 'Wynd',
 'Cul-de-s

In [73]:
all_items = []
for section, items in section_dict.items():
    for index, item in enumerate(items):
        #print(item)
        item = re.sub(r'\[|\]', '', item)
        if '<ref' in item:
            #print(item)
            all_items.append(re.search(r'(.*)\<ref', item).group(1))
        elif '|' in item:
            #print(item)
            all_items.append(re.search(r'\|(.*)', item).group(1))
        elif ' and' in item:
            print(f"### and-split {index}")
            if ',' in item:
                #print(f"comma-split{item.split(',')}")
                item = re.sub(r'and', ',', item)
                #print(f"### comma-split {index}")
                for ytem in item.split(','):
                    all_items.append(ytem.strip())
            else:
                #print(f"and-split{item.split('and')}")
                #print(f"### and-split {index}")
                for ytem in item.split(' and'):
                    all_items.append(ytem.strip())
        elif ' &' in item:
            for ytem in item.split(' &'):
                    all_items.append(ytem.strip())

        else:
            all_items.append(item)
all_items

['California State Route 85',
 'California State Route 87',
 'California State Route 130',
 'State',
 'The Alameda',
 'Alum Rock Avenue',
 'Asbury',
 'Arroyo de Oro',
 'Auzerais Avenue',
 'Bailey Avenue',
 'Balbach',
 'Balboa Avenue',
 'Bascom Avenue',
 'Bassett Street',
 'Bernal Road',
 'Berryessa Road',
 'Bird Avenue',
 'Blach Place',
 'Blossom Hill Road',
 'Bollinger Road',
 'Booksin Avenue',
 'Branham Lane',
 'Cahill Street',
 'Camden Avenue',
 'Canoas Garden Avenue',
 'Chaboya Road',
 'Chapman Street',
 'Cinnabar Street',
 'Cleaves Avenue',
 'Cleveland Avenue',
 'Coe Avenue',
 'Cory Avenue',
 'Cottle Avenue (not to be confused with Cottle Road)',
 'Cottle Road',
 'Cunningham Avenue',
 'Curtner Avenue',
 'Dana Avenue',
 'Davis Street',
 'De Mattei Court',
 'Delmas Avenue (downtown San Jose)',
 'Di Fiore Drive',
 'Donohue Drive',
 'Dry Creek Road',
 'Eastus Drive',
 'Emory Street',
 'Fiesta Lane',
 'Flickinger Avenue',
 'Fowler Road',
 'Fox Avenue',
 'Forbes Drive',
 'Le Franc Drive

In [74]:
for index, item in enumerate(all_items):
    #remove digits
    item = re.sub('\d', '', item).strip()
    # remove comments in parens
    item = re.sub(r'\((.*)\)', '', item).strip()
    #check if last word is street type
    last_word = item.split(' ')[-1]
    if last_word in street_types:
        print(last_word)
        print(item)
        all_items[index] = item.replace(last_word, '').strip()
    else:
        all_items[index] = item
        
        #print(item)

Route
California State Route
Route
California State Route
Route
California State Route
Avenue
Alum Rock Avenue
Avenue
Auzerais Avenue
Avenue
Bailey Avenue
Avenue
Balboa Avenue
Avenue
Bascom Avenue
Street
Bassett Street
Road
Bernal Road
Road
Berryessa Road
Avenue
Bird Avenue
Place
Blach Place
Road
Blossom Hill Road
Road
Bollinger Road
Avenue
Booksin Avenue
Lane
Branham Lane
Street
Cahill Street
Avenue
Camden Avenue
Avenue
Canoas Garden Avenue
Road
Chaboya Road
Street
Chapman Street
Street
Cinnabar Street
Avenue
Cleaves Avenue
Avenue
Cleveland Avenue
Avenue
Coe Avenue
Avenue
Cory Avenue
Avenue
Cottle Avenue
Road
Cottle Road
Avenue
Cunningham Avenue
Avenue
Curtner Avenue
Avenue
Dana Avenue
Street
Davis Street
Court
De Mattei Court
Avenue
Delmas Avenue
Drive
Di Fiore Drive
Drive
Donohue Drive
Road
Dry Creek Road
Drive
Eastus Drive
Street
Emory Street
Lane
Fiesta Lane
Avenue
Flickinger Avenue
Road
Fowler Road
Avenue
Fox Avenue
Drive
Forbes Drive
Drive
Le Franc Drive
Street
Fremont Street
Av

In [75]:
dedupe_all_items = []
for item in all_items:
    if item not in dedupe_all_items and:
        dedupe_all_items.append(item)
dedupe_all_items

['California State',
 'State',
 'The Alameda',
 'Alum Rock',
 'Asbury',
 'Arroyo de Oro',
 'Auzerais',
 'Bailey',
 'Balbach',
 'Balboa',
 'Bascom',
 'Bassett',
 'Bernal',
 'Berryessa',
 'Bird',
 'Blach',
 'Blossom Hill',
 'Bollinger',
 'Booksin',
 'Branham',
 'Cahill',
 'Camden',
 'Canoas Garden',
 'Chaboya',
 'Chapman',
 'Cinnabar',
 'Cleaves',
 'Cleveland',
 'Coe',
 'Cory',
 'Cottle',
 'Cunningham',
 'Curtner',
 'Dana',
 'Davis',
 'De Mattei',
 'Delmas',
 'Di Fiore',
 'Donohue',
 'Dry Creek',
 'Eastus',
 'Emory',
 'Fiesta',
 'Flickinger',
 'Fowler',
 'Fox',
 'Forbes',
 'Le Franc',
 'Fremont',
 'Gion',
 'Gish',
 'Goodwin',
 'Goodyear',
 'Graham',
 'Hamilton',
 'Hamline',
 'Hanchett',
 'Harding',
 'Harwood',
 'Hedding',
 'Hensley',
 'Hester',
 'Hobson',
 'Hoover',
 'Idaho',
 'Jackson',
 'Julian',
 'Keeble',
 'King',
 'Lawrence',
 'Leigh',
 'Leland',
 'Lenzen',
 'Lester',
 'Lightstone',
 'Lindbergh',
 'Locust',
 'Luther',
 'Magellan',
 'Magnolia',
 'Mariposa',
 'Martin',
 'Maybury',
 'M

In [77]:
# remove empty string if caused eg 'Pellier Avenue & Court'
dedupe_all_items = [item for item in dedupe_all_items if item != '']
dedupe_all_items

['California State',
 'State',
 'The Alameda',
 'Alum Rock',
 'Asbury',
 'Arroyo de Oro',
 'Auzerais',
 'Bailey',
 'Balbach',
 'Balboa',
 'Bascom',
 'Bassett',
 'Bernal',
 'Berryessa',
 'Bird',
 'Blach',
 'Blossom Hill',
 'Bollinger',
 'Booksin',
 'Branham',
 'Cahill',
 'Camden',
 'Canoas Garden',
 'Chaboya',
 'Chapman',
 'Cinnabar',
 'Cleaves',
 'Cleveland',
 'Coe',
 'Cory',
 'Cottle',
 'Cunningham',
 'Curtner',
 'Dana',
 'Davis',
 'De Mattei',
 'Delmas',
 'Di Fiore',
 'Donohue',
 'Dry Creek',
 'Eastus',
 'Emory',
 'Fiesta',
 'Flickinger',
 'Fowler',
 'Fox',
 'Forbes',
 'Le Franc',
 'Fremont',
 'Gion',
 'Gish',
 'Goodwin',
 'Goodyear',
 'Graham',
 'Hamilton',
 'Hamline',
 'Hanchett',
 'Harding',
 'Harwood',
 'Hedding',
 'Hensley',
 'Hester',
 'Hobson',
 'Hoover',
 'Idaho',
 'Jackson',
 'Julian',
 'Keeble',
 'King',
 'Lawrence',
 'Leigh',
 'Leland',
 'Lenzen',
 'Lester',
 'Lightstone',
 'Lindbergh',
 'Locust',
 'Luther',
 'Magellan',
 'Magnolia',
 'Mariposa',
 'Martin',
 'Maybury',
 'M

In [114]:
# make 1 list
#all_items = [item for section, items in section_dict.items() for item in items]
# make 2 lists, nested
# all_items = [item for section, item in section_dict.items()]
# flatten lists 2 ways
# all_items = [lyst1 for lyst2 in all_items for lyst1 in lyst2]
#all_items = [item for items in all_items for item in items]
all_items 

['Abbey Orchard',
 'Abingdon',
 'Ambrosden',
 'Apsley',
 'Arneway',
 'Artillery',
 'Artillery',
 'Ashley',
 'Atterbury',
 'Barton',
 "Bennett's Yard",
 'Bessborough',
 'Birdcage',
 'Bloomburg',
 'Bressenden',
 "Brewer's",
 'Bridge',
 'Broad Sanctuary',
 'Little Sanctuary',
 'The Sanctuary',
 'Broadway',
 'Buckingham',
 'Buckingham',
 'Buckingham',
 'Buckingham Palace',
 'Palace',
 'Palace',
 'Bulinga',
 'Butler',
 'Canon',
 'Cardinal',
 'Carey',
 'Carlisle',
 'Carteret',
 'Castle',
 'Cathedral',
 'Catherine',
 'Causton',
 'Caxton',
 'Chadwick',
 'Chapter',
 'Charing Cross',
 'Coburg',
 'Cockpit Steps',
 'Constitution',
 'Cowley',
 "Craig's",
 'Cureton',
 'Dacre',
 'Dartmouth',
 'Dean Bradley',
 'Dean Farrar',
 'Dean Ryle',
 'Dean Stanley',
 'Dean Trench',
 "Dean's Yard",
 "Little Dean's Yard",
 'Derby',
 'Douglas',
 'Downing',
 'Duke of Wellington',
 'Elizabeth Court',
 'Elverton',
 'Emery Hill',
 'Erasmus',
 'Esterbrooke',
 'Francis',
 'Fynes',
 'Gayfere',
 'Great College',
 'College'

In [78]:
limit = 'all'
with open(f"../output/{topic}_{subtopic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in dedupe_all_items:
        fp.write(item + '\n')

In [ ]:
# use if items all are/begin with list of links [[link]]

items = []
descriptions = []
with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines[77:291]):
        
        if re.match('\*\s*\[\[', line[0:5]):
            #print("%%%Match%%%")
            #print(line)
            try:
                line = re.sub(r'\<ref.*?\</ref>|\<ref.*\/>', '', line)
                
            except:
                topic_error_line = f"{topic} line sub parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                item = re.search(r'\[\[(.*?)\]\]', line).group(1)
                
            except:
                topic_error_line = f"{topic} item parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            try:
                
                description = re.search(r'\]\]\:*(.*)', line).group(1)
                
                
            except:
                topic_error_line = f"{topic} description parse error"
                item_line = f"{str(index)}, {line}"
                print(log_error(topic_error_line, item_line))
                
            items.append(item)
            descriptions.append(description)
        else:
            continue
            #print("%%%No%%%")
            #print(line)

In [ ]:
items = [re.sub(r'\:wiktionary\:', '', item) for item in items]
items = [re.sub(r'\|.*', '', item) for item in items]
items

In [ ]:
# use to find info page for each item/link and rank 
item_infos = []
for index, title in enumerate(items):
    with open(topic + '_error_log.txt', 'w+', encoding="utf-8") as filepath:
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
        
        item_infos.append({
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        })


In [ ]:
# transfer item_infos to json file
import json
with open(topic + '_infos.json', 'w') as filepath:
    json.dump(item_infos, filepath)

In [ ]:
error_items = [item[topic] for item in item_infos if item['views'] == '#error#']

for index, title in enumerate(error_items):
    with open('info_error_log.txt', 'w+', encoding="utf-8") as filepath:
        #print(title)
        del_idx = 0
        for index, item in enumerate(item_infos):
            if item[topic] == title:
                del_idx = index
                #del item_infos[index]
        #print(item)
        
        
        if re.match(r'\W*', title):
            print('%%%')
            search = re.search(r'(\w+)?', title).group(1)
            print(search)
            title = search
        
        print(title)
        
        url = 'https://en.wikipedia.org/w/index.php?title=' + title + '&action=info'

        #print(url)
        response = requests.get(url)
        doc = lxml.html.fromstring(response.content)
        soup = BeautifulSoup(response.content, 'html.parser')
  
        try:
            # info_table = doc.xpath('//*[@class="wikitable mw-page-info"]')[0]
            watchers = doc.xpath('//*[@id="mw-pageinfo-watchers"]')[0][1].text
            
        except:
            topic_error_line = f"watchers parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            watchers = "#error#"

        try:
            redirects = doc.xpath('//*[contains(text(),"redirects")]/../../td')[1].text
            #redirects = doc.xpath('//*[contains(text(),"redirects")]/parent::node()/parent::node()')
            #redirects = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[2]/tr[9]/td')[1].text
            
        except:
            topic_error_line = f"redirects parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            redirects = "#error#"
             
        try:
            views = doc.xpath('//*[@id="mw-pvi-month-count"]/td/div')[0].text
            
        except:
            topic_error_line = f"views parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            views = "#error#"        
        
        try:
            edits = doc.xpath('//*[@id="mw-pageinfo-edits"]/td')[1].text
            #edits = doc.xpath('/html/body/div[3]/div[3]/div[3]/table[4]/tr[5]/td')[1].text
    
        except:
            topic_error_line = f"edits parse error"
            item_line = f"{str(index)}, {title}, {url}"
            print(log_error(topic_error_line, item_line))
            filepath.write(str(index) + ' : ' + title + ' : ' + url + '\n')
            edits = "#error#"
            
        item_infos[del_idx] = {
            topic: title,
            'watchers': watchers,
            'redirects': redirects,
            'views': views,
            'edits': edits
        }

In [ ]:
with open('mineral_infos.json', 'r') as jfile:
    data=jfile.read()

# parse file
item_infos = json.loads(data)

In [ ]:
_item_infos = item_infos
len(_item_infos)

In [ ]:
#item_infos = [{item['views']: int(item['views'].replace(',', ''))} for item in item_infos]
for index, item in enumerate(_item_infos):
    if item['views'] != '#error#':
        if item['watchers'] == 'Fewer than 30 watchers':
            item['watchers'] = '29'
        item['views'] = int(item['views'].replace(',', ''))
        item['watchers'] = int(item['watchers'].replace(',', ''))
        #item['edits'] = int(item['edits'].replace(',', ''))
        
        
        

In [ ]:
by_views = sorted(_item_infos, key=lambda k : k['views'], reverse=True)
for index, item in enumerate(by_views):
    item['view_rank'] = index

In [ ]:
by_edits = sorted(_item_infos, key=lambda k : k['edits'], reverse=True)
for index, item in enumerate(by_edits):
    item['edit_rank'] = index

In [ ]:
by_watchers = sorted(_item_infos, key=lambda k : k['watchers'], reverse=True)
for index, item in enumerate(by_watchers):
    item['watcher_rank'] = index

In [ ]:
for item in _item_infos:
    #item['total_rank'] = (item['view_rank'] + item['edit_rank'] + item['watcher_rank']) / 3
    item['total_rank'] = (item['view_rank'] + item['watcher_rank']) / 2

    

In [ ]:
by_total = sorted(_item_infos, key=lambda k : k['total_rank'], reverse=True)

In [ ]:
dedupe = []
for item in by_total:
    if item[topic] not in dedupe:
        dedupe.append(item[topic])
        

In [ ]:
start = 0
limit = 44
top_items = dedupe[start:limit]

In [ ]:
with open(f"{topic}_{limit}.txt", 'w+', encoding="utf-8") as fp:
    for item in top_items:
        fp.write(item + '\n')

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['total_rank'], ascending=False).head(100)

In [ ]:
min_df = pd.DataFrame(_item_infos)
min_df.head()
min_df.sort_values(by=['views'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['watchers'], ascending=False).head(100)

In [ ]:
min_df.sort_values(by=['edits'], ascending=False).head(100)

In [ ]:
# parse data

lines = textDATA["parse"]["wikitext"]["*"].split('\n')[3:-10] # [::-1]
this_sec = {}
sec = ''
sub_sec = ''
with open('error_lines.txt', 'w+', encoding="utf-8") as filepath:
    for index, line in enumerate(lines):
        if index == 23:
            print(line)
        if re.match('==\w', line[0:3]):
            sec = line.strip().replace("=", "")
            if len(this_sec) > 0:
                section_dict[secs[secs.index(sec) - 1]] = this_sec
            this_sec = {}
        if type(section_dict[sec]).__name__ == 'list':
            if re.match('\*\[\[', line[0:3]):
                try:
                    poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                    #print(poet)

                except:
                    print("##########################")
                    print("#### poet parse error ####")
                    print(poet_method, index, line)
                    filepath.write(line + '\n')
                    print(line[-2:])
                    print("##########################")

                date_m = re.search('\((.*)\)', line)

                try:
                    date = date_m.group(0)
                except:
                    print("##########################")
                    print("#### date parse error ####")
                    print(index, line)
                    filepath.write(line + '\n')
                    date = ""
                    print("##########################")
                try:
                    # description = re.search('\)*,*.*', line).group(1)
                    # description = re.search('\),+.*', line).group(1)
                    # description = re.search('\]\s.*\)(.*)', line).group(1)
                    description = re.search('\]\](\)*,*.*)', line).group(1)
                except Exception as e:
                    print("#################################")
                    print("#### description parse error ####")
                    print(e)
                    print(index, line)
                    filepath.write(line + '\n')
                    print("#################################")
                # print(this_sec)
                # print(sub_sec)
                section_dict[sec].append({
                    'poet': poet,
                    'date': date,
                    'description': description
                })
        else:
            #print(sec)
            if re.match('===\w', line[0:4]):
                sub_sec = line.strip().replace("=", "")
                #print(sub_sec)
            if line.strip().replace("=", "") in section_dict[sec]:
                #print(sub_sec)
                this_sec[sub_sec] = []
            if sub_sec != '':
                if re.match('\*\[\[', line[0:3]):
                    try:
                        poet = re.search(r'\[\[(.*?)\]\]', line).group(1)
                        #print(poet)
                    except:
                        print("##########################")
                        print("#### poet parse error ####")
                        print(index, line)
                        filepath.write(line + '\n')
                        print(line[-2:])
                        print("##########################")
                    date_m = re.search('\((.*)\)', line)
                    try:
                        date = date_m.group(0)
                    except Exception as e:
                        print("##########################")
                        print("#### date parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        date = ""
                        print("##########################")
                    try:
                        # description = re.search('\)*,*.*', line).group(1)
                        # description = re.search('\),+.*', line).group(0)
                        # description = re.search('\]\s.*\)(.*)', line).group(0)
                        description = re.search('\]\](\)*,*.*)', line).group(1)

                    except Exception as e:
                        print("#################################")
                        print("#### description parse error ####")
                        print(e)
                        print(index, line)
                        filepath.write(line + '\n')
                        description = ""
                        print("#################################")

                    this_sec[sub_sec].append({
                        'poet': poet,
                        'date': date,
                        'description': description
                    })
